# Perzeptron

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


Ein paar Hilfsfunktionen:

In [ ]:
from numpy.random import default_rng
rng = default_rng(888)

# generate fake data that is linearly separable with a margin epsilon given the data
def getfake(samples, dimensions, epsilon):
    wfake = rng.standard_normal(size=(dimensions))   # fake weight vector for separation
    bfake = rng.standard_normal(size=(1))            # fake bias
    wfake = wfake / np.linalg.norm(wfake)                 # rescale to unit length

    # making some linearly separable data, simply by chosing the labels accordingly
    X = np.zeros(shape=(samples, dimensions))
    Y = np.zeros(shape=(samples))

    i = 0
    while (i < samples):
        tmp = rng.standard_normal(size=(1,dimensions))
        margin = np.dot(tmp, wfake) + bfake
        if (np.linalg.norm(tmp) < 3) & (abs(margin) > epsilon):
            X[i,:] = tmp[0]
            Y[i] = np.sign(margin)
            i += 1
    return X, Y

# plot the data with colors chosen according to the labels
def plotdata(X,Y):
    plt.scatter(X[:, 0], X[:, 1], c=Y, alpha=0.7)

# plot contour plots on a [-3,3] x [-3,3] grid
def plotscore(w,d):
    xgrid = np.arange(-3, 3, 0.02)
    ygrid = np.arange(-3, 3, 0.02)
    xx, yy = np.meshgrid(xgrid, ygrid)
    zz = np.zeros(shape=(xgrid.size, ygrid.size, 2))
    zz[:,:,0] = np.array(xx)
    zz[:,:,1] = np.array(yy)
    vv = np.dot(zz,w) + d
    CS = plt.contour(xgrid,ygrid,vv)
    plt.clabel(CS, inline=1, fontsize=10)

X, Y = getfake(50, 2, 0.3)

Plotten wir unsere synthetischen, linear separierbaren Daten:

In [ ]:
plotdata(X,Y)
plt.show()

Jetzt definieren wir unser Perzeptron:

$$\hat{y}=sgn(w^T x + b)$$

$\hat{y}$ ist positiv oder negativ je nachdem, ob x ober- oder unterhalb der durch $(w, b)$ definierten Gerade ist.

In [ ]:
def perceptron(w, b, idx, x, y):
    y_hat = np.sign(np.dot(w,x) + b)
    if idx >= 0: print(f'Datensatz({idx}):   {x}, label {y}')
    if (y * y_hat) <= 0:
        if idx >= 0: print(f'Anpassung da falsche Klassifikation, sollte {y} sein')
        w += y * x
        b += y
        if idx >= 0: print(f'Neue Weight {w}, Bias  {b}')
        return 1
    else:
        if idx >= 0: print('keine Anpassung, richtig klassifiziert')
        return 0

    ... und schicken es auf die Reise ...

In [ ]:
w = np.zeros(shape=(2))
b = np.zeros(shape=(1))
for idx, (x,y) in enumerate(zip(X,Y)):
    res = perceptron(w,b,idx, x,y)
    if (res == 1):
        plotscore(w,b)
        plotdata(X,Y)
        plt.scatter(x[0], x[1], color='g', s=100)
        plt.show()


# The Perceptron Convergence Theorem

Das Theorem besagt, dass, wenn alle Samples innerhalb eines Kreises mit Radius R liegen und die Punkte mit einem Abstand von mindestens $\epsilon$ linear separierbar sind, dann konvergiert das Updateverfahren und benötigt maximal $\frac{2(R^2+1)}{\epsilon^2}$ Updates.

In [ ]:
Eps = np.arange(0.025, 2., 0.05)
Err = np.zeros(shape=(Eps.size))

n = 10 # Anzahl Durchläufe

for j in range(n):
    for (i,epsilon) in enumerate(Eps):
        X, Y = getfake(500, 2, epsilon)

        for (x,y) in zip(X,Y):
            Err[i] += perceptron(w,b,-1, x,y)

Err = Err / n

fig, ax = plt.subplots(1, figsize=(8,8))
ax.plot(Eps, Err, label='Durchschnittliche Anzahl an Updates für Training')
ax.plot(Eps, 20/(Eps**2), label='Theoretisch maximale Anzahl an Updates')
ax.set_ylim([0, 35])
ax.legend();
